## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-12-13-05-09-11 +0000,bbc,Witkoff to meet Zelensky for latest Ukraine wa...,https://www.bbc.com/news/articles/c0l9954yr9ko...
1,2025-12-13-04-58-26 +0000,nypost,"‘Pulp Fiction,’ ‘The Mask’ actor Peter Greene ...",https://nypost.com/2025/12/12/us-news/peter-gr...
2,2025-12-13-04-35-10 +0000,nyt,"After False Earthquake Alert in Nevada, Lawmak...",https://www.nytimes.com/2025/12/12/us/usgs-nev...
3,2025-12-13-04-20-41 +0000,startribune,"Wild trade Marco Rossi, Zeev Buium, Liam Ohgre...",https://www.startribune.com/minnesota-wild-qui...
4,2025-12-13-04-13-00 +0000,wsj,Fighting Persists on Thai-Cambodian Border Aft...,https://www.wsj.com/world/fighting-persists-on...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2355/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
47,trump,44
121,new,17
185,year,13
148,venezuela,10
25,was,10


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
122,2025-12-12-21-34-00 +0000,wsj,President Trump said he was leaning toward cho...,https://www.wsj.com/economy/central-banking/tr...,123
158,2025-12-12-19-33-00 +0000,wsj,New Epstein Photos Released by Democrats Show ...,https://www.wsj.com/politics/democrats-release...,96
25,2025-12-13-02-16-51 +0000,bbc,"New photos from Epstein estate show Trump, And...",https://www.bbc.com/news/articles/crk777lj763o...,95
219,2025-12-12-15-51-53 +0000,nypost,"Woody Allen, Andrew Mountbatten-Windsor, Bill ...",https://nypost.com/2025/12/12/us-news/woody-al...,94
61,2025-12-13-00-06-15 +0000,nyt,New Photos From Epstein Estate Show Ties to Tr...,https://www.nytimes.com/2025/12/12/us/politics...,93


## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
122,123,2025-12-12-21-34-00 +0000,wsj,President Trump said he was leaning toward cho...,https://www.wsj.com/economy/central-banking/tr...
158,52,2025-12-12-19-33-00 +0000,wsj,New Epstein Photos Released by Democrats Show ...,https://www.wsj.com/politics/democrats-release...
308,46,2025-12-12-09-08-17 +0000,nyt,Venezuela’s Nobel Winner Will Go to Norway for...,https://www.nytimes.com/2025/12/06/world/ameri...
18,37,2025-12-13-02-37-44 +0000,nypost,Homeless woman accused of stabbing tourist ins...,https://nypost.com/2025/12/12/us-news/homeless...
129,37,2025-12-12-21-03-00 +0000,wsj,Reddit Sues Australian Government for Under-16...,https://www.wsj.com/business/media/reddit-sues...
42,34,2025-12-13-01-12-49 +0000,nypost,White House hosts ‘Miracle on Ice’ heroes as T...,https://nypost.com/2025/12/12/us-news/hockey-h...
74,33,2025-12-12-23-35-04 +0000,nypost,Alleged California ‘cult’ leaders arrested for...,https://nypost.com/2025/12/12/us-news/alleged-...
191,33,2025-12-12-17-34-00 +0000,wsj,Amtrak managers are giving up $16 million of t...,https://www.wsj.com/politics/policy/amtrak-wor...
163,32,2025-12-12-19-14-51 +0000,bbc,EU backs indefinite freeze on Russia's frozen ...,https://www.bbc.com/news/articles/c98nnd01g91o...
188,31,2025-12-12-17-46-17 +0000,nypost,Disney CEO Bob Iger raises red flags about Net...,https://nypost.com/2025/12/12/business/disney-...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
